In [4]:
from glob import glob
import os
import pandas as pd
import numpy as np
import errno
from tqdm import tqdm
import sys
import shutil
sys.path.append('./modified_medsam_repo')

In [3]:
listo1 = glob('/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/dir_structure_for_yolov7/train/images/*.png')
listo2 = glob('/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/dir_structure_for_yolov7/val/images/*.png')
listo3 = glob('/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/dir_structure_for_yolov7/test/images/*.png')


In [4]:
fracs = [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 75, 100]

for frac in fracs:
    print(frac)
    df = pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/path_df_constant_bbox.csv')
    #split = pd.read_pickle('/gpfs/data/luilab/karthik/pediatric_seg_proj/train_val_test_split.pickle')
    split = pd.read_pickle(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/{frac}.pkl')

    lookup = {}
    for x in split['train']:
        lookup[x] = 'train'
    for x in split['val']:
        lookup[x] = 'val'
    for x in split['test']:
        lookup[x] = 'test'

    listos = {'train':[], 'val':[], 'test':[]}
    for i, r in df.iterrows():
        if r['id'] in lookup:
            tag = lookup[r['id']]
        else:
            continue
        path = f'/gpfs/data/cbi/hcp/hcp_ya/hcp_ya_slices_npy/dir_structure_for_yolov7/{tag}/images/{r["id"]}_slice{r["slice"]}.png'
        listos[tag].append(path)
    rt_path = f'/gpfs/data/luilab/karthik/pediatric_seg_proj/yolov10_txt_files/{frac}'
    os.makedirs(rt_path)
    for tag in ['train', 'val', 'test']:
        targ = f'{rt_path}/{tag}.txt'
        with open(targ, 'w') as f:
            for line in listos[tag]:
                f.write(f'{line}\n')

0.1
0.5
1
2.5
5
10
25
50
75
100


In [9]:
# write yaml files
targ = '/gpfs/home/kn2347/yolov10/data'
src = '/gpfs/data/luilab/karthik/pediatric_seg_proj/yolov10_txt_files'

for frac in [0.1, 0.5, 1, 2.5, 5, 10, 25, 50, 75, 100]:
    to_dup_file = f'{targ}/hcp.yaml'
    write_file = f'{targ}/hcp_subset_{frac}.yaml'

    shutil.copy(to_dup_file, write_file)
    
    dicto = {}
    for tag in ['train', 'val', 'test']:
        dicto[tag] = f'{src}/{frac}/{tag}.txt'
    

    with open(write_file, 'r') as file:
        # read a list of lines into data
        newdata = file.readlines()
    
    # edit the lines
    newdata[1] = f'train: /gpfs/data/luilab/karthik/pediatric_seg_proj/yolov10_txt_files/{frac}/train.txt\n'
    newdata[2] = f'val: /gpfs/data/luilab/karthik/pediatric_seg_proj/yolov10_txt_files/{frac}/val.txt\n'
    newdata[3] = f'test: /gpfs/data/luilab/karthik/pediatric_seg_proj/yolov10_txt_files/{frac}/test.txt\n'

    # write it back
    with open(write_file, 'w') as file:
        file.writelines(newdata )


### Old stuff

In [ ]:
filelist = glob('/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/*.csv')
for x in filelist:
    fileid = os.path.basename(x).split('.csv')[0]
    to_path = os.path.join('/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/dir_structure_for_yolov7/subsample_symlinks', fileid)
    if not os.path.exists(os.path.join(to_path, 'train', 'images')):
        os.makedirs(os.path.join(to_path, 'train', 'images'))
        os.makedirs(os.path.join(to_path, 'train', 'labels'))

    df_read = pd.read_csv(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/{fileid}.csv')
    df_read = df_read.drop_duplicates(subset = ['id', 'slice'])
    print(f'on {fileid}')
    for idx, r in tqdm(df_read.iterrows(), total = df_read.shape[0]):
        ending = f'{r.id}_slice{r.slice}.png'
        from_path = os.path.join('/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/dir_structure_for_yolov7/train/images', ending)
        to_path_im = os.path.join(to_path, 'train', 'images', ending)
        try:
            os.symlink(from_path, to_path_im)
        except OSError as e:
            if e.errno == errno.EEXIST:
                os.remove(to_path_im)
                os.symlink(from_path, to_path_im)
            else:
                print('uncaught error')

        # label
        ending = f'{r.id}_slice{r.slice}.txt'
        from_path = os.path.join('/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/dir_structure_for_yolov7/train/labels', ending)
        to_path_la = os.path.join(to_path, 'train', 'labels', ending)
        try:
            os.symlink(from_path, to_path_la)
        except OSError as e:
            if e.errno == errno.EEXIST:
                os.remove(to_path_la)
                os.symlink(from_path, to_path_la)
            else:
                print('uncaught error')



In [14]:
filelist = glob('/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/*.csv')


for x in filelist:
    listo_paths = []
    fileid = os.path.basename(x).split('.csv')[0]
    #to_path = os.path.join('/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/dir_structure_for_yolov7/subsample_symlinks', fileid)

    df_read = pd.read_csv(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/{fileid}.csv')
    df_read = df_read.drop_duplicates(subset = ['id', 'slice'])
    print(f'on {fileid}')
    for idx, r in tqdm(df_read.iterrows(), total = df_read.shape[0]):
        ending = f'{r.id}_slice{r.slice}.png'
        from_path = os.path.join('/gpfs/data/luilab/karthik/pediatric_seg_proj/hcp_ya_slices_npy/dir_structure_for_yolov7/train/images', ending)
        listo_paths.append(from_path)

    with open(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/yolov7_subsample_txt_files/{fileid}.txt', 'w') as fp:
        for item in listo_paths:
            # write each item on a new line
            fp.write("%s\n" % item)
        print(f'for {fileid}: {len(listo_paths)}')
        


        



on 500


100%|██████████| 78117/78117 [00:04<00:00, 15778.81it/s]


for 500: 78117
on 100


100%|██████████| 28133/28133 [00:01<00:00, 15050.42it/s]


for 100: 28133
on 750


...

for 50: 21723
on 5


100%|██████████| 16121/16121 [00:00<00:00, 17423.66it/s]


for 5: 16121
on 10


100%|██████████| 16663/16663 [00:01<00:00, 14727.26it/s]

for 10: 16663


In [15]:
np.unique(pd.read_csv('/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/50.csv')['id'].to_numpy()).shape[0] - 111

44

In [15]:
for ext in ['5', '10', '50', '100', '250', '500']:
    print(pd.read_csv(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/yolov7_subsample_txt_files/{ext}.txt').shape[0])
    xd = pd.read_csv(f'/gpfs/data/luilab/karthik/pediatric_seg_proj/subset_train_id_dfs_pooled/{ext}.csv')
    xd = xd.drop_duplicates(subset = ['id', 'slice'])
    print(xd.shape[0])
    #print(xd)
    print('===')


16120
16121
===
16662
16663
===
21722
21723
===
28132
28133
===
46824
46825
===
78116
78117
===
